<span style="font-size:12px">

#### ▶ 자연어 감성분석
- 감성사전 기반 : 미리 정의된 감성 단어 사전 사용해서 단어별 감정 점수를 모아 판단 (규칙기반) 
    - TextBlob, AFINN, VADER
    - 빠르게 큰 텍스트 스트림(예: 트위터, 페이스북 댓글)을 실시간으로 모니터링할 때.
    - 라벨이 없는 데이터에서 빠른 탐색(what’s the overall sentiment?)이 필요할 때.
    - 리소스(라벨링 시간)가 부족하거나 실시간 처리가 중요한 경우.
- 머신러닝 기반 : 텍스트를 벡터화(TF-IDF 등)하고 분류기(예: Naive Bayes, SVM, 로지스틱 회귀, 신경망)를 학습 (통계기반)
    - TF-IDF 벡터화
    - 선형회귀, 로지스틱회귀
    - F1 Score, Recison, Recall -> classification report
    - 도메인 특화 언어(제품 리뷰, 의료 문서 등)에서 높은 정확도가 필요할 때.
    - 라벨(학습 데이터)을 확보할 수 있고 모델 튜닝/검증을 할 수 있을 때.

<span style="font-size:12px">

#### ▶ 사용 데이터
- NLTK 영화 리뷰 (2000개)
- 다음 영화 리뷰


<span style="font-size:12px">

#### ▶ 알고리즘
- TextBlob 사전기반 감성분석
- AFINN 감정 점수 맵핑
- VADER(Valence Aware Dictionary) 소셜미디어 최적화 감성분석
- TF-IDF 텍스트 벡터화
-  Multinomial Naive Bayes 확률 기반 분류

<span style="font-size:12px">

#### ▶ TextBlob
- 이 영화는 정말 좋고 재미있다
    - 좋다 +1 (긍정)
    - 재미있다 +1 (긍정)
    - +2 > 0 ----> 긍정(pos) 분류
- polarity (극성도) = (긍정단어 개수 - 부정단어 개수) / 전체단어개수
- -1.0 ~ +1.0
- 0 은 중립
- Subjectivity (주관성) 평가대상 단어 비율
- 0.0 ~ 1.0
- 0 : 객관적  / 1 : 주관적
- 문맥무시하고 단어 극성만 고려
- 이 영화는 나쁘지 않다 ---> 나쁘다(-) 않다(-) 로 인식
- 단점 : 빠른 속도, 학습 불필요
- 사용 : 실시간 감성분석, 스트리밍 데이터

In [ ]:
# %conda install textBlob

3 channel Terms of Service accepted
Channels:
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from textblob import TextBlob, Word
text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)
print(blob.sentences)  #문장 추출
print(blob.words)  #단어 추출
print(blob.tags)   #품사 추출

# import nltk
# nltk.download('brown')
print(blob.noun_phrases)  #명사 추출


print('===========감성분석===========')
blob.sentiment


[Sentence("TextBlob is amazingly simple to use."), Sentence("What a wonderful library for NLP!")]
['TextBlob', 'is', 'amazingly', 'simple', 'to', 'use', 'What', 'a', 'wonderful', 'library', 'for', 'NLP']
[('TextBlob', 'NNP'), ('is', 'VBZ'), ('amazingly', 'RB'), ('simple', 'JJ'), ('to', 'TO'), ('use', 'VB'), ('What', 'WP'), ('a', 'DT'), ('wonderful', 'JJ'), ('library', 'NN'), ('for', 'IN'), ('NLP', 'NNP')]
['textblob', 'wonderful library', 'nlp']
===========감성분석===========


Sentiment(polarity=0.5, subjectivity=0.6785714285714286)

<span style="font-size:12px"> 
==========================================

#### ▶ AFINN (Lexicon-Based)
- 각 단어의 -5 ~ +5 의 점수를 부여하고 합산
- 이 영화는 좋지만 좋지 않은 부분도 있다
    - 좋다 +3 / 좋지 +3 / 나쁘다 -3 ----> +3 > 0 긍정
- score = sum(word_sentiment_value)
    - 분류규칙 score > 0 긍정 score < 0 부정
- 이모티콘 지원
- 강도표현 인식 very, really 등

- 강조 수정자 (intensifiers)
    - 매우좋다 = 1.5x (좋다의 점수)

- AFINN vs TextBlob
    - AFINN : 더 정확할 점수 매핑
    - TextBlob : 더 일반적인 접근

In [ ]:
%pip install afinn

In [ ]:
from afinn import Afinn
af = Afinn()
text1 = "TextBlob is amazingly simple to use."
text2 = "What a wonderful library for NLP!"
score1 = af.score(text1)
score2 = af.score(text2)
score1, score2


(0.0, 4.0)

<span style="font-size:12px"> 
==========================================

#### ▶ VADER  소셜 미디어 텍스트에 최적화
- 이 영화는 정말정말 훌륭해 !!
- 훌륭하다 (기본) +0.7  / 정말정말 (강조) x1.5
- !!!(문장부호강종) x1.2
- 4개의 감정 지수
    - positive 긍정 확률 0 ~ 1
    - negative 부정 확률
    - neutral 중립 확률
    - compound 종합점수 -1 ~ 1
- score = compound_score / sqrt(compound_score**2 + 0.0625)   #0.0625 : 고정값?
    - score >= 0.05 긍정
    - score <= -0.05 부정
    - 그 사이는 중립
- 대소문자 구분 AMAZING amazing 다른 점수를 가짐
- :-) 긍정 /  :-( 부정

In [9]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\playdata2\AppData\Roaming\nltk_data...


True

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyer = SentimentIntensityAnalyzer()
sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]
for s in sentences:
    scores = analyer.polarity_scores(s)
    print(f' 문장: {s}, \n 점수: {scores}')

 문장: I love this product! It's absolutely amazing 😍, 
 점수: {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
 문장: This is the worst movie I've ever seen..., 
 점수: {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
 문장: The food was okay, not great but not bad either., 
 점수: {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
 문장: I’m REALLY happy with the results!!!, 
 점수: {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
 문장: Not good at all. I’m disappointed., 
 점수: {'neg': 0.579, 'neu': 0.421, 'pos': 0.0, 'compound': -0.6711}


In [ ]:
import nltk
from nltk.corpus import movie_reviews
from textblob import TextBlob
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import pandas as pd

# nltk 데이터 다운로드
nltk.download('movie_reviews', quiet=True)
nltk.download('vader_lexicon', quiet=True)


In [38]:
# 영화 리뷰 데이터 로드
fileids = movie_reviews.fileids()

reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]


# 속도를 위해 데이터 한정
# reviews = [movie_reviews.raw(fileid) for fileid in fileids[:50]] + \
#         [movie_reviews.raw(fileid) for fileid in fileids[-50:]]
# categories = [movie_reviews.categories(fileid)[0] for fileid in fileids[:50]]+\
#         [movie_reviews.categories(fileid)[0] for fileid in fileids[-50:]]

# 리뷰 총개수 / 긍정 부정 리뷰 갯수
len(reviews), categories.count('pos'), categories.count('neg')

(2000, 1000, 1000)

In [39]:
#### 1 TextBlob
def sentiment_textblob(docs):
    return ['pos' if TextBlob(doc).sentiment.polarity>0 else 'neg' for doc in docs]
pred_textblob = sentiment_textblob(reviews)
accuracy_textblob = accuracy_score(categories, pred_textblob)
print ( f' 정확도 : {accuracy_textblob: .2f}')

 정확도 :  0.60


In [46]:
#### 2 AFINN
def sentiment_afinn(docs):
    afn = Afinn (emoticons=True)
    return [ 'pos' if afn.score(doc)>0 else 'neg' for doc in docs]
pred = sentiment_afinn(reviews)
accuracy = accuracy_score(categories, pred)
print ( f' 정확도 : {accuracy: .2f}')

 정확도 :  0.66


In [47]:
#### 3 VADER
def sentiment_vader(docs):
    vad = SentimentIntensityAnalyzer()
    return ['pos' if vad.polarity_scores(doc) ['compound']>0 else 'neg' for doc in docs]
pred = sentiment_vader(reviews)
accuracy = accuracy_score(categories, pred)
print ( f' 정확도 : {accuracy: .2f}')

 정확도 :  0.64


In [27]:
##### 머신러닝 기반 감성분석  / 딥러닝 해보세여 - MLP ? softmax, crossentropy, loss, entropy, 과적합 등 확인
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [28]:
# 나이브 베이즈
# 베이즈 정리
# "좋다" 단어를 본 후 이 리뷰가 긍정일 확률
# p(긍정 | "좋다") = p("좋다" | 전체 긍정) x p(긍정) / p("좋다")

In [45]:
# 데이터 분할
dataset = train_test_split(reviews, categories, test_size=0.2, random_state=42, stratify=categories)
len(dataset[0]), len(dataset[2]), len(dataset[1]), len(dataset[3])


# 인덱스 변수 이름	설명
#   0	X_train	학습용 입력 데이터
#   1	X_test	테스트용 입력 데이터
#   2	y_train	학습용 라벨
#   3	y_test	테스트용 라벨

(1600, 1600, 400, 400)

In [41]:
# tf-idf 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
x_train = vectorizer.fit_transform(dataset[0])
x_test = vectorizer.transform(dataset[1])
y_train =dataset[2]
y_test = dataset[3]
x_train.shape

(1600, 1000)

In [42]:
### 1. MNB
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train,y_train)
pred = mnb_clf.predict (x_test)
print (classification_report (y_test,pred))

              precision    recall  f1-score   support

         neg       0.83      0.81      0.82       200
         pos       0.81      0.83      0.82       200

    accuracy                           0.82       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.82      0.82      0.82       400



In [43]:
### logistic regression
lr = LogisticRegression()
lr.fit(x_train,y_train)
pred = lr.predict (x_test)
print (classification_report (y_test,pred))

              precision    recall  f1-score   support

         neg       0.84      0.81      0.82       200
         pos       0.81      0.85      0.83       200

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [70]:
# 성능향상 - 전처리
# 소문자변환 - 연속된 문자열 중에 3글자 이상 - 어간추출(형태소 분석) - 불용어 제거
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

def custom_tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[\w']{3,}")
    tokens = tokenizer.tokenize(text)
    porter = PorterStemmer()
    stop_words = stopwords.words('english')
    return [porter.stem(token) for token in tokens if token not in stop_words]

vector = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    max_features=1000,
    min_df=5,
    max_df=0.5,
    ngram_range=(1,3)
)

x_train = vector.fit_transform(dataset[0])
x_test = vector.transform(dataset[1])

c:\Users\playdata2\miniconda3\envs\LLM\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [53]:
x_train
x_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.03746766, 0.        ,
        0.03645065],
       [0.06204417, 0.05176801, 0.        , ..., 0.04337023, 0.        ,
        0.04219301],
       [0.        , 0.        , 0.        , ..., 0.0768619 , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06536913, 0.        , 0.        , ..., 0.04569445, 0.        ,
        0.08890828],
       [0.10722645, 0.        , 0.        , ..., 0.07495363, 0.        ,
        0.        ]], shape=(1600, 1000))

In [78]:
# 모델 함수
def evaluate_model(model):
    model.fit(x_train,y_train)
    pred = model.predict (x_test)
    print (f' {str(model).split("(")[0]} 모델:\n{classification_report (y_test,pred)}')

In [79]:
### 전처리 후 모델 평가
evaluate_model(LogisticRegression())  #상기 전처리에서 ngram 추가후 lr모델에서 0.01 상승
evaluate_model(MultinomialNB())


 LogisticRegression 모델:
              precision    recall  f1-score   support

         neg       0.87      0.80      0.84       200
         pos       0.82      0.89      0.85       200

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400

 MultinomialNB 모델:
              precision    recall  f1-score   support

         neg       0.80      0.82      0.81       200
         pos       0.82      0.80      0.81       200

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.81      0.81      0.81       400



In [80]:
# 데이터 - 토큰화 & 점수 인코딩 - 시퀀스 패딩
# baseline 일반 신경망 : Dense
# simple RNN : 기울기 소실 문제
# Bidirectional LSTM : 양방향 학습

In [81]:
# Tokenization 텍스트를 숫자로 변환
# Word Embedding 단어를 고차원 벡터로 변환
# Sequence Padding 길이가 다른 문장을 같은 크기로 고정
# simple RNN
# LSTM  : RNN의 경사소실문제를 해결
# Bidiretional LSTM : 양방향 컨텐츠 학습

In [82]:
# 토큰화 : 숫자로 변환하는 과정
# 토크나이저 3단계
    # 1. fit_on_text (texts) 가장 빈도가 높은 단어의 인덱스를 구축해서 딕셔너리로 반환
    # 2. texts_to_sequence (texts) 각문서를 정수 시퀀스로 반환
    # 3. pad_sequence() 길이를 정규화(같은 길이)

In [83]:
# 딥러닝에서 워드 임베딩 레이어 : 각 단어를 고정된 크기의 실수 벡터

In [ ]:
# %pip install tensorflow
# %pip install torch

In [32]:
# 파이토치 버전 (책은 tensorflow)

# 토큰화
from collections import Counter
import numpy as np
import torch 

sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [33]:
# 단어 분할 및 빈도 계산
all_words = []
for i in [review.split() for review in sample_reviews]:
    all_words.extend(i)

In [14]:
# 단어빈도
word_freq = Counter(all_words)

In [ ]:
# 1. Tokenizer 구현
class SimpleTokenizer :
    def __init__ (self, num_words=10, oov_token = 'UNK'):
        self.num_words = num_words  #단어를 빈도수 기준으로 몇개까지 쓸지 (가장 자주 등장한 단어만 남김)
        self.oov_token = oov_token  #사전에 없는 단어(Out-Of-Vocabulary, OOV)를 대체할 특수 토큰 이름 (‘UNK’ = unknown)
        self.word_index = {}
        self.index_word = {}


        # oov_token 설명
        # tokenizer.fit_on_texts(["i love apple"])
        # tokenizer.texts_to_sequences(["i love banana"])
        # → "banana"는 사전에 없죠? -> OOV토큰 -> "banana"처럼 사전에 없는 단어는 "UNK"라는 특수 단어로 대체



    # 단어사전 만들기 : 문장에서 단어를 추출하고, 빈도 높은 순서로 인덱스 부여
    def fit_on_texts(self, texts):
        '''문장을 단어 인덱스로 반환'''
        all_words = []
        for i in [review.split() for review in sample_reviews]:
            all_words.extend(i)
        word_freq = Counter(all_words)
        
        # oov 토큰을 1로 설정
        self.word_index[self.oov_token] = 1  # 'UNK' (사전에 없는 단어용) → 1번으로 설정
        self.index_word[1] = self.oov_token
        # 나머지 단어는 많이 등장한 순서대로 2,3,4,... 순서
        idx = 2
        for word, _ in word_freq.most_common(self.num_words-1) :
            self.word_index[word] = idx
            self.index_word[idx] = word
            idx += 1
                        # most_common() : Counter 객체의 내장 함수
                        # 👉 등장 빈도가 높은 순서대로 단어와 빈도를 튜플 형태로 반환
                        # 예. ord_freq.most_common(2) --> 가장 자주 나온 단어 2개만 반환
                        # most_common(n) → 가장 자주 나온 n개 단어를 반환

                        # word_freq.most_common(self.num_words-1) 
                        # 위에 oov_token 이 1번 자리르 차지하고 있어서 -1로 계산함 (OOV토큰 제외)
                        

    # 텍스트를 숫자로 변환
    # 예시
    # {'UNK': 1, 'i': 2, 'movie': 3, 'love': 4, 'hate': 5}
    # texts = ["i love movie", "i like movie"]
    # tokenizer.texts_to_sequences(texts)
    # [[2, 4, 3], [2, 1, 3]]
    def texts_to_sequences (self, texts):
        '''텍스트를 정수 시퀀스로 변환'''
        sequences = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocabulary에 있으면 인덱스를 사용, 없으면 oov
                word_index = self.word_index.get(word,1)
                seq.append(word_index)
            sequences.append(seq)
        return sequences
                                            # 딕셔너리.get()문법
                                            # dict.



# Tokenizer 생성 및 학습
tokenizer = SimpleTokenizer(num_words=10, oov_token='UNK')
tokenizer.fit_on_texts(sample_reviews)
tokenizer.word_index

{'UNK': 1,
 'movie': 2,
 'great': 3,
 'wonderful': 4,
 'this': 5,
 'is': 6,
 'and': 7,
 'bad': 8,
 'with': 9,
 'poor': 10}

In [39]:
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
sequences

[[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]

In [42]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다
def pad_sequence_manual (sequences, max_len=10, padding='pre', value=0):
    '''패딩구현'''
    padded = []
    for seq in sequences:
        if len(seq) >= max_len :
            if padded == 'pre':
                padded_seq = seq[-max_len]
            else :
                padded_seq = seq [:max_len]
        else :
            pad_length = max_len-len(seq)
            if padding == 'pre':
                padded_seq = [value]*pad_length + seq
            else :
                padded_seq = seq + [value]*pad_length
        padded.append(padded_seq)
    return np.array(padded)
padded = pad_sequence_manual(sequences)
padded



array([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
       [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
       [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [43]:
# pytorch tensor 변환
sequences_tensor = torch.LongTensor(padded)
sequences_tensor

tensor([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
        [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
        [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [45]:
# 2. 워드 임베딩
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.nn as nn

print(f' 패딩된 시퀀스 형태 : {sequences_tensor.shape}')
print (f' 첫번째 : {sequences_tensor[0]}')

# pytorch embedding 레이어 생성
    # num_ebeddings : 각 단어를 몇차원 벡터로 표현할 것인지
    # padding_idx 같이 맞출때 채우는 값
embedding_layer = nn.Embedding(num_embeddings=1000, embedding_dim=8, padding_idx=0)
embedded = embedding_layer(sequences_tensor)
print(f'입력형태 : {sequences_tensor.shape}')
print(f'출력형태 : {embedded.shape}')


 패딩된 시퀀스 형태 : torch.Size([3, 10])
 첫번째 : tensor([0, 0, 0, 0, 5, 2, 6, 3, 7, 4])
입력형태 : torch.Size([3, 10])
출력형태 : torch.Size([3, 10, 8])


In [46]:
# 임베딩 벡터 상세 분석
# 샘플 데이터의 첫 3개단어 임베딩
for word_idx in range(3):
    embedding_vec = embedded[0,word_idx].detach().numpy()
    word_id = sequences_tensor[0, word_idx].item()
    print(f' 단어 idf {word_id}: {embedding_vec} -- 8차원 중 처음 4개')

 단어 idf 0: [0. 0. 0. 0. 0. 0. 0. 0.] -- 8차원 중 처음 4개
 단어 idf 0: [0. 0. 0. 0. 0. 0. 0. 0.] -- 8차원 중 처음 4개
 단어 idf 0: [0. 0. 0. 0. 0. 0. 0. 0.] -- 8차원 중 처음 4개


In [48]:
# 임베딩 행렬
embedding_matrix = embedding_layer.weight.detach().numpy()
print (f'임베딩 행렬 형태 : {embedding_matrix.shape}')
print (f'패딩(id=0)의 임베딩 {embedding_matrix[0]}')
print (f'단어 id=5 {embedding_matrix[5]}')

임베딩 행렬 형태 : (1000, 8)
패딩(id=0)의 임베딩 [0. 0. 0. 0. 0. 0. 0. 0.]
단어 id=5 [ 0.43877324 -1.3464957   0.95847356 -1.3694234  -0.8483197  -1.313673
  0.08624984  0.55934185]


In [ ]:
# ---

In [49]:
# Rnn 적용
class RnnModule(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim ):
        super(RnnModule, self).__init__()
        self.embdding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    def forward (self, x):
        x_emb = self.embdding(x)  #(batch, seq_len, embedding_dim) 출력
        rnn_out, h_n = self.rnn(x_emb)  
            # rnn_out(batch, seq_len, hidden_dim)
            # h_n(1, batch, hidden_dim) 

        #마지막 스텝의 출력
        last_output = rnn_out[:, -1, :]  #(batch, hidden_dim) 출력
        output = self.sigmoid(self.fc(last_output))
        return output